### Table of Contents

* [Initial Configurations](#IC)
    * [Import Libraries](#IL)
    * [Autheticate the AML Workspace](#AML)
* [Get Data (Bronze)](#GD)
    * [Setup Directory Structure](#SD)
    * [Get Wiki data](#WD)
    * [Parse Wiki data](#PD)
    * [Write the RAW parsed data](#WD)
    * [Check the uploaded files](#UF)
    * [Read the wiki country and language data](#MD)
    * [Features](#F)    
* [Data Wrangling](#DW)
    * [Step1. Rename Columns to a standard format](#S1)
    * [Step2. Remove rows based on value of certain features](#S2)
    * [Step3. Fix column types](#S3)
    * [Step4. Handle null values](#S4)
    * [Step5. Check for duplicates and handle if present](#S5)
    * [Step6. Text data clean (NLP): Clean up the string attributes](#S6)
    * [Step7. Drop the old columns, rename the cleaned columns](#S7)
    * [Step8. Fix casing of Country column](#S8)
    * [Step9. Fix null columns and data type after the new column additions](#S9)
* [Put Data (Silver)](#PS)
    * [Write the transformed dataframe to the silver zone](#WS)
    * [Read silver zone country and language file](#RS)


### Initial Configurations <a class="anchor" id="IC"></a>

#### Import Libraries <a class="anchor" id="IL"></a>

In [5]:
#Import required Libraries
import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from bs4 import BeautifulSoup
import urllib3

import azureml.core
import azureml.automl
#from azureml.core.experiment import Experiment
from azureml.core import Workspace, Dataset, Datastore

#### Autheticate the AML Workspace <a class="anchor" id="AML"></a>

In [2]:
#Autheticate the AML Workspace
workspace = Workspace.from_config()
output = {}
output['Subscription ID'] = workspace.subscription_id
output['Workspace Name'] = workspace.name
output['Resource Group'] = workspace.resource_group
output['Location'] = workspace.location
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Subscription ID,7ca151c5-e4f7-4663-9583-834f4e0e6ed4
Workspace Name,houston-techsummit-workspace
Resource Group,calcutta_demos
Location,eastus2


### Get Data (Bronze) <a class="anchor" id="GD"></a>
- Raw data extraction for the file, API based and web datasets. Let us call this __Bronze Layer__.
- Data transformation using python from Raw to Processed stage. We will call this __Silver Layer__.
- Finally store the processed data using standard taxonomy in a SQL based serving layer. We will call this __Gold Layer__.

#### Setup Directory Structure <a class="anchor" id="SD"></a>

In [3]:
data_folder = os.path.join(os.getcwd(), 'data')
#'/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-cpu-ds12-v2/code/Users/rabiswas/Movies_Data_Consortium/data'

#Create the data directory
os.makedirs(data_folder, exist_ok=True)

#Create the bronze, silver and gold folders
bronze_data_folder = data_folder +"/bronze"
os.makedirs(bronze_data_folder, exist_ok=True)

silver_data_folder = data_folder +"/silver"
os.makedirs(silver_data_folder, exist_ok=True)

gold_data_folder = data_folder +"/gold"
os.makedirs(gold_data_folder, exist_ok=True)


#Create sub folder
#'/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-cpu-ds12-v2/code/Users/rabiswas/Movies_Data_Consortium/data/bronze/flat_file'
file_data_bronze = bronze_data_folder +"/website_file"
os.makedirs(file_data_bronze, exist_ok=True)

#Manually upload the folders from Kaggle

In [4]:
file_data_bronze

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-cpu-ds12-v2/code/Users/rabiswas/Movies_Data_Consortium/data/bronze/website_file'

#### Get Wiki data <a class="anchor" id="WD"></a>

In [7]:
#Make a http get request to the wiki
#We need a PoolManager instance to make requests. 
#This object handles all of the details of connection pooling and thread safety so that you don’t have to:
http = urllib3.PoolManager()

#To make a request use request():
r = http.request('GET', 'https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory')
#r.data

In [124]:
#r.data

#### Parse Wiki data <a class="anchor" id="PD"></a>

In [11]:
#Read the html payload using BeautifulSoup
# query the website and return the html to the variable ‘page’
page = r.data

#Finally, parse the page into BeautifulSoup format so we can use BeautifulSoup to work on it.
# parse the html using beautiful soup and store in variable 'soup'
soup = BeautifulSoup(page, 'html.parser')

In [12]:
#Find the table structure you will need to deal with (how many tables there are?) 
all_tables = soup.find_all("table")
print("Total number of tables are {} ".format(len(all_tables)))

Total number of tables are 9 


In [14]:
#Find the right table using bs4 
# We used the  the class attribute
data_table = soup.find('table', attrs={'class': 'wikitable'})
print(type(data_table))

<class 'bs4.element.Tag'>


In [125]:
#data_table

In [20]:
#The <table> tag defines an HTML table.

#Each table row is defined with a <tr> tag. Each table header is defined with a <th> tag. 
#Each table data

#data_table

#<table class="wikitable sortable">
#<tbody><tr>
#<th><span class="anchor" id="A"></span>Country
#</th>
#<th width="300pt">Official language
#</th>
#<th width="300pt">Regional language
#</th>
#<th>Minority language
#</th>
#<th>National language
#</th>
#<th>Widely spoken
#</th></tr>
#<tr>
#<td><span class="flagicon"><img alt="" class="thumbborder" data-file-height="300" data-file-width="600" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/7/7a/Flag_of_the_Republic_of_Abkhazia.svg/23px-Flag_of_the_Republic_of_Abkhazia.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/7/7a/Flag_of_the_Republic_of_Abkhazia.svg/35px-Flag_of_the_Republic_of_Abkhazia.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/7/7a/Flag_of_the_Republic_of_Abkhazia.svg/46px-Flag_of_the_Republic_of_Abkhazia.svg.png 2x" width="23"/></span> <a class="mw-redirect" href="/wiki/Languages_of_Abkhazia" title="Languages of Abkhazia">Abkhazia</a><sup class="reference" id="cite_ref-DT_1-0"><a href="#cite_note-DT-1">[a]</a></sup>
#</td>
#<td>
#<ul><li><a href="/wiki/Abkhaz_language" title="Abkhaz language">Abkhaz</a></li>
#<li><a href="/wiki/Russian_language" title="Russian language">Russian</a></li></ul>
#</td>
#<td>
#</td>
#<td><a href="/wiki/Georgian_language" title="Georgian language">Georgian</a>
#</td>
#<td><a href="/wiki/Abkhaz_language" title="Abkhaz language">Abkhaz</a>
#</td>
#<td>
#</td></tr>

In [32]:
#Get the first row since that contains the header of the table
header_table = data_table.tbody.findAll('tr', recursive=False)[0]
header_table

<tr>
<th><span class="anchor" id="A"></span>Country
</th>
<th width="300pt">Official language
</th>
<th width="300pt">Regional language
</th>
<th>Minority language
</th>
<th>National language
</th>
<th>Widely spoken
</th></tr>

In [34]:
#Extract the header from the table
header = [th.getText().strip() for th in header_table.findAll('th')]
header

['Country',
 'Official language',
 'Regional language',
 'Minority language',
 'National language',
 'Widely spoken']

In [ ]:
data = data_table.tbody.findAll('tr', recursive=False)

In [46]:
td = data[1]
td

<tr>
<td><span class="flagicon"><img alt="" class="thumbborder" data-file-height="300" data-file-width="600" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/7/7a/Flag_of_the_Republic_of_Abkhazia.svg/23px-Flag_of_the_Republic_of_Abkhazia.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/7/7a/Flag_of_the_Republic_of_Abkhazia.svg/35px-Flag_of_the_Republic_of_Abkhazia.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/7/7a/Flag_of_the_Republic_of_Abkhazia.svg/46px-Flag_of_the_Republic_of_Abkhazia.svg.png 2x" width="23"/></span> <a class="mw-redirect" href="/wiki/Languages_of_Abkhazia" title="Languages of Abkhazia">Abkhazia</a><sup class="reference" id="cite_ref-DT_1-0"><a href="#cite_note-DT-1">[a]</a></sup>
</td>
<td>
<ul><li><a href="/wiki/Abkhaz_language" title="Abkhaz language">Abkhaz</a></li>
<li><a href="/wiki/Russian_language" title="Russian language">Russian</a></li></ul>
</td>
<td>
</td>
<td><a href="/wiki/Georgian_language"

In [47]:
td.getText().strip()

'Abkhazia[a]\n\n\nAbkhaz\nRussian\n\n\n\nGeorgian\n\nAbkhaz'

In [48]:
td.getText().strip().split("\n\n", 5)

['Abkhazia[a]', '\nAbkhaz\nRussian', '', 'Georgian', 'Abkhaz']

In [49]:
td = data[2]
td.getText().strip().split("\n\n", 5)

['Afghanistan[1]',
 '\nPashto\nDari',
 '\nUzbek[b]\nTurkmen[b]\nPashai[b]\nNuristani[b]\nBalochi[b]\nPamiri[b]',
 '',
 'Pashto']

In [52]:
data_rows = []
#Loop from 1st row since 0th row is header
for i in range (1,len(data)):
    td = data[i]
    data_rows.append(td.getText().strip().split("\n\n", 5))
data_rows[3]

['Algeria[3]', '\nArabic\nTamazight', '', '', '\nArabic\nTamazight', 'French']

In [53]:
df_website_data = pd.DataFrame(data_rows, columns=header)
df_website_data.head()

,Country,Official language,Regional language,Minority language,National language,Widely spoken
0,Abkhazia[a],\nAbkhaz\nRussian,,Georgian,Abkhaz,None
1,Afghanistan[1],\nPashto\nDari,\nUzbek[b]\nTurkmen[b]\nPashai[b]\nNuristani[b]\nBalochi[b]\nPamiri[b],,Pashto,None
2,Albania[2],Albanian,,\nGreek\nMacedonian\nAromanian,,Italian
3,Algeria[3],\nArabic\nTamazight,,,\nArabic\nTamazight,French
4,Andorra,Catalan[4],,\nSpanish\nFrench\nPortuguese,None,None


In [69]:
#Validate that we got all the data
df_website_data.tail(2)

,Country,Official language,Regional language,Minority language,National language,Widely spoken
232,Zambia,English,None,None,None,None
233,Zimbabwe,\nChewa\nChibarwe\nEnglish\nKalanga\nKhoisan[g][77]\nNambya\nNdau\nNdebele\nShangani\nShona\nSign language[h]\nSotho\nTonga\nTswana\nVenda\nXhosa,None,None,None,None


#### Write the RAW parsed data <a class="anchor" id="WRD"></a>

In [57]:
bronze_file_name = file_data_bronze + "/wiki_country_language.csv"
df_website_data.to_csv(bronze_file_name, index=False)

In [58]:
bronze_file_name

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-cpu-ds12-v2/code/Users/rabiswas/Movies_Data_Consortium/data/bronze/website_file/wiki_country_language.csv'

#### Check the uploaded files <a class="anchor" id="UF"></a>

In [59]:
#List the folder structure and the files
for root, directories, files in os.walk(file_data_bronze, topdown=True):
	for name in directories:
		print(os.path.join(root, name))    
	for name in files:
		print(os.path.join(root, name))

/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-cpu-ds12-v2/code/Users/rabiswas/Movies_Data_Consortium/data/bronze/website_file/wiki_country_language.csv


#### Read the wiki country and language data <a class="anchor" id="MD"></a>

In [61]:
#Read the wiki country and language data

df_wiki_country_language_bronze = pd.read_csv(bronze_file_name)
df_wiki_country_language_bronze.head()

,Country,Official language,Regional language,Minority language,National language,Widely spoken
0,Abkhazia[a],\nAbkhaz\nRussian,NaN,Georgian,Abkhaz,NaN
1,Afghanistan[1],\nPashto\nDari,\nUzbek[b]\nTurkmen[b]\nPashai[b]\nNuristani[b]\nBalochi[b]\nPamiri[b],NaN,Pashto,NaN
2,Albania[2],Albanian,NaN,\nGreek\nMacedonian\nAromanian,NaN,Italian
3,Algeria[3],\nArabic\nTamazight,NaN,NaN,\nArabic\nTamazight,French
4,Andorra,Catalan[4],NaN,\nSpanish\nFrench\nPortuguese,NaN,NaN


In [62]:
#Number of rows and columns
df_wiki_country_language_bronze.shape

(234, 6)

In [64]:
#It is easier to view the data if we transpose
df_wiki_country_language_bronze.head(3).transpose()


,0,1,2
Country,Abkhazia[a],Afghanistan[1],Albania[2]
Official language,\nAbkhaz\nRussian,\nPashto\nDari,Albanian
Regional language,NaN,\nUzbek[b]\nTurkmen[b]\nPashai[b]\nNuristani[b]\nBalochi[b]\nPamiri[b],NaN
Minority language,Georgian,NaN,\nGreek\nMacedonian\nAromanian
National language,Abkhaz,Pashto,NaN
Widely spoken,NaN,NaN,Italian


In [65]:
#List of columns
df_wiki_country_language_bronze.columns

Index(['Country', 'Official language', 'Regional language',
       'Minority language', 'National language', 'Widely spoken'],
      dtype='object')

#### Features <a class="anchor" id="F"></a>
- __Country:__ Indicates the country.
- __Official language:__ one designated as having a unique legal status in the state, typically, the language used in a nation's legislative bodies, and often, official government business
- __Regional language:__ one designated as having official status limited to a specific area, administrative division, or territory of the state (on this page a regional language will have parentheses next to it that contain a region, province, etc. where the language has regional status)
- __Minority language:__ (as used here) one spoken by a minority population within the state and officially designated as such; typically afforded protection and designated an officially permissible language for legal and government business in a specific area or territory of the state (on this page a minority language will be followed by parentheses that identify its minority status)
- __National language:__ one that uniquely represents the national identity of a state, nation, and/or country and so designated by a country's government; some are technically minority languages (on this page a national language will be followed by parentheses that identify it as a national language status). Some countries have more than one language with this status

In [70]:
df_wiki_country_language_bronze.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Country            234 non-null    object
 1   Official language  234 non-null    object
 2   Regional language  63 non-null     object
 3   Minority language  54 non-null     object
 4   National language  89 non-null     object
 5   Widely spoken      56 non-null     object
dtypes: object(6)
memory usage: 11.1+ KB


Next we will start cleaning up the data.

### Data Wrangling <a class="anchor" id="DW"></a>

#### Step1.  Rename Columns to a standard format  <a class="anchor" id="S1"></a>

- __Official language__ will become __official_language__
- __Regional language__ will become __regional_language__
- __Minority language__ will become __minority_language__
- __National language__ will become __national_language__
- __Widely spoken__ will become __widely_spoken__

In [87]:
#Replace header
df_wiki_country_language_bronze.rename(columns={"Official language": "official_language", "Regional language": "regional_language", \
                                                "Minority language": "minority_language", "National language": "national_language", \
                                                "Widely spoken": "widely_spoken"}, inplace=True)

In [88]:
#List of columns
df_wiki_country_language_bronze.columns

Index(['Country', 'official_language', 'regional_language',
       'minority_language', 'national_language', 'widely_spoken'],
      dtype='object')

#### Step2.  Remove rows based on value of certain features <a class="anchor" id="S2"></a>

In [74]:
df_wiki_country_language_bronze.tail()

,Country,official_language,regional_language,minority_language,national_language,Widely spoken
229,Country,Official language,Regional language,Minority language,National language,Widely spoken
230,Yemen,Arabic,NaN,NaN,NaN,NaN
231,Country,Official language,Regional language,Minority language,National language,Widely spoken
232,Zambia,English,NaN,NaN,NaN,NaN
233,Zimbabwe,\nChewa\nChibarwe\nEnglish\nKalanga\nKhoisan[g][77]\nNambya\nNdau\nNdebele\nShangani\nShona\nSign language[h]\nSotho\nTonga\nTswana\nVenda\nXhosa,NaN,NaN,NaN,NaN


As we can see in row 229 we have values on Country as "Country". This is because wiki broke the main table into several tables. Lets drop these rows.

In [80]:
#We have 23 junk values
omit_values_list = ['Country']
df_wiki_country_language_bronze[df_wiki_country_language_bronze.Country.isin(omit_values_list)].value_counts()

Country  official_language  regional_language  minority_language  national_language  Widely spoken
Country  Official language  Regional language  Minority language  National language  Widely spoken    23
dtype: int64

In [81]:
#Get the omit values in a list and then filter
df_wiki_country_language_bronze = df_wiki_country_language_bronze[~df_wiki_country_language_bronze.Country.isin(omit_values_list)]

In [82]:
#We have removed the bad rows succesfully
df_wiki_country_language_bronze[df_wiki_country_language_bronze.Country.isin(omit_values_list)].value_counts()

Series([], dtype: int64)

#### Step3.  Fix column types <a class="anchor" id="S3"></a>

In [89]:
df_wiki_country_language_bronze.dtypes

Country              object
official_language    object
regional_language    object
minority_language    object
national_language    object
widely_spoken        object
dtype: object

In [90]:
df_wiki_country_language_bronze['Country'] = pd.Series(df_wiki_country_language_bronze['Country'], dtype="string")
df_wiki_country_language_bronze['official_language'] = pd.Series(df_wiki_country_language_bronze['official_language'], dtype="string")
df_wiki_country_language_bronze['regional_language'] = pd.Series(df_wiki_country_language_bronze['regional_language'], dtype="string")
df_wiki_country_language_bronze['minority_language'] = pd.Series(df_wiki_country_language_bronze['minority_language'], dtype="string")
df_wiki_country_language_bronze['national_language'] = pd.Series(df_wiki_country_language_bronze['national_language'], dtype="string")
df_wiki_country_language_bronze['widely_spoken'] = pd.Series(df_wiki_country_language_bronze['widely_spoken'], dtype="string")

In [91]:
df_wiki_country_language_bronze.dtypes

Country              string
official_language    string
regional_language    string
minority_language    string
national_language    string
widely_spoken        string
dtype: object

#### Step4.  Handle null values <a class="anchor" id="S4"></a>

In [92]:
df_wiki_country_language_bronze.isnull().sum()

Country              0  
official_language    0  
regional_language    171
minority_language    180
national_language    145
widely_spoken        178
dtype: int64

In [93]:
#Lets fill the NAs with an empty string '' for regional_language, minority_language, national_language, widely_spoken attribute
df_wiki_country_language_bronze[['regional_language']] = df_wiki_country_language_bronze[['regional_language']].fillna(value='')
df_wiki_country_language_bronze[['minority_language']] = df_wiki_country_language_bronze[['minority_language']].fillna(value='')
df_wiki_country_language_bronze[['national_language']] = df_wiki_country_language_bronze[['national_language']].fillna(value='')
df_wiki_country_language_bronze[['widely_spoken']] = df_wiki_country_language_bronze[['widely_spoken']].fillna(value='')

In [94]:
#Checking and there are no null values in the dataset
df_wiki_country_language_bronze.isnull().sum()

Country              0
official_language    0
regional_language    0
minority_language    0
national_language    0
widely_spoken        0
dtype: int64

#### Step5.  Check for duplicates and handle if present <a class="anchor" id="S5"></a>

In [95]:
#Duplicate rows
df_wiki_country_language_bronze[df_wiki_country_language_bronze.duplicated()].head()

,Country,official_language,regional_language,minority_language,national_language,widely_spoken


There are no duplicate rows in he dataset, so we are good on this front.

#### Step6.  Text data clean (NLP): Clean up the string attributes <a class="anchor" id="S6"></a>

In [96]:
#Build punctuation dictionary
import unicodedata
import sys

# Create a dictionary of punctuation characters
punctuation = dict.fromkeys(i for i in range(sys.maxunicode)
                            if unicodedata.category(chr(i)).startswith('P'))
# Add the backtick/ Grave accent character
punctuation.update({96:None})

In [97]:
#Let us clean up and make the data ready
# As we are going to use words as features so we can use some text formatting techniques which will help us in feature extraction
#  including removing punctuation marks/digits ,and also stop-words. In addition, the implementation of lemmatization words using NLTK
#       Tokenization is the last step to break reviews up into words and other meaningful tokens.

import re 
import string
#pip install nltk
import nltk
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer

from nltk.corpus import stopwords 
ENGLISH_STOP_WORDS = stopwords.words('english')

def function_clean_stop(text):
    #convert into lowercase
    text = text.lower()

    #removing the URL Http
    text = re.sub(r"http\S+", "", text) 

    # Removal of mentions
    #text = re.sub("@[^\s]*", "", text) -> Taken care if in the punctuations. But I want to keep the word.

    # Removal of hashtags
    #text = re.sub("#[^\s]*", "", text) -> Taken care if in the punctuations. But I want to keep the word.

    # Removal of numbers
    text = re.sub('[0-9]*[+-:]*[0-9]+', '', text)
    text = re.sub("'s", "", text)   

    #remove all punctuation from the text.
    text = str(text.translate(punctuation))
    
    listofwords = text.strip().split()          # to remove any space from beginning and the end of text
    tokenized_words = []    
    for word in listofwords:
        if not word in ENGLISH_STOP_WORDS:
            lemm_word = WordNetLemmatizer().lemmatize(word)
            if len(lemm_word)>0:
                tokenized_words.append(lemm_word)
    return_str = ' '.join([str(elem) for elem in tokenized_words])             
    return(return_str)

In [98]:
df_wiki_country_language_bronze.head()

,Country,official_language,regional_language,minority_language,national_language,widely_spoken
0,Abkhazia[a],Abkhaz Russian,,Georgian,Abkhaz,
1,Afghanistan[1],Pashto Dari,Uzbek[b] Turkmen[b] Pashai[b] Nuristani[b] Balochi[b] Pamiri[b],,Pashto,
2,Albania[2],Albanian,,Greek Macedonian Aromanian,,Italian
3,Algeria[3],Arabic Tamazight,,,Arabic Tamazight,French
4,Andorra,Catalan[4],,Spanish French Portuguese,,


In [100]:
# applying the cleaning function to Country column
df_wiki_country_language_bronze['country_cleaned'] = df_wiki_country_language_bronze['Country'].apply(lambda overview: function_clean_stop(overview))

In [101]:
df_wiki_country_language_bronze[['Country','country_cleaned']].head()

,Country,country_cleaned
0,Abkhazia[a],abkhaziaa
1,Afghanistan[1],afghanistan
2,Albania[2],albania
3,Algeria[3],algeria
4,Andorra,andorra


In [103]:
# applying the cleaning function to the official_language column
df_wiki_country_language_bronze['official_language_cleaned'] = df_wiki_country_language_bronze['official_language'].apply(lambda overview: function_clean_stop(overview))
df_wiki_country_language_bronze[['official_language','official_language_cleaned']].head()

,official_language,official_language_cleaned
0,Abkhaz Russian,abkhaz russian
1,Pashto Dari,pashto dari
2,Albanian,albanian
3,Arabic Tamazight,arabic tamazight
4,Catalan[4],catalan


In [104]:
# applying the cleaning function to the regional_language column
df_wiki_country_language_bronze['regional_language_cleaned'] = df_wiki_country_language_bronze['regional_language'].apply(lambda overview: function_clean_stop(overview))
df_wiki_country_language_bronze[['regional_language','regional_language_cleaned']].head()

,regional_language,regional_language_cleaned
0,,
1,Uzbek[b] Turkmen[b] Pashai[b] Nuristani[b] Balochi[b] Pamiri[b],uzbekb turkmenb pashaib nuristanib balochib pamirib
2,,
3,,
4,,


In [106]:
# applying the cleaning function to the minority_language column
df_wiki_country_language_bronze['minority_language_cleaned'] = df_wiki_country_language_bronze['minority_language'].apply(lambda overview: function_clean_stop(overview))
df_wiki_country_language_bronze[['minority_language','minority_language_cleaned']].head()

,minority_language,minority_language_cleaned
0,Georgian,georgian
1,,
2,Greek Macedonian Aromanian,greek macedonian aromanian
3,,
4,Spanish French Portuguese,spanish french portuguese


In [108]:
# applying the cleaning function to the national_language column
df_wiki_country_language_bronze['national_language_cleaned'] = df_wiki_country_language_bronze['national_language'].apply(lambda overview: function_clean_stop(overview))
df_wiki_country_language_bronze[['national_language','national_language_cleaned']].head()

,national_language,national_language_cleaned
0,Abkhaz,abkhaz
1,Pashto,pashto
2,,
3,Arabic Tamazight,arabic tamazight
4,,


In [109]:
# applying the cleaning function to the widely_spoken column
df_wiki_country_language_bronze['widely_spoken_cleaned'] = df_wiki_country_language_bronze['widely_spoken'].apply(lambda overview: function_clean_stop(overview))
df_wiki_country_language_bronze[['widely_spoken','widely_spoken_cleaned']].head()

,widely_spoken,widely_spoken_cleaned
0,,
1,,
2,Italian,italian
3,French,french
4,,


#### Step7.  Drop the old columns, rename the cleaned columns <a class="anchor" id="S7"></a>

In [111]:
df_wiki_country_language_bronze.columns

Index(['Country', 'official_language', 'regional_language',
       'minority_language', 'national_language', 'widely_spoken',
       'country_cleaned', 'official_language_cleaned',
       'regional_language_cleaned', 'minority_language_cleaned',
       'national_language_cleaned', 'widely_spoken_cleaned'],
      dtype='object')

In [112]:
#Removed the duplicated features
df_wiki_country_language_bronze.drop(['Country', 'official_language', 'regional_language','minority_language', 'national_language', 'widely_spoken'], axis=1, inplace=True)

In [113]:
#Rename the columns
df_wiki_country_language_bronze.rename(columns={"official_language_cleaned": "official_language", "regional_language_cleaned": "regional_language", \
                                                "minority_language_cleaned": "minority_language", "national_language_cleaned": "national_language", \
                                                "widely_spoken_cleaned": "widely_spoken", "country_cleaned": "country"}, inplace=True)

In [114]:
df_wiki_country_language_bronze.columns

Index(['country', 'official_language', 'regional_language',
       'minority_language', 'national_language', 'widely_spoken'],
      dtype='object')

In [115]:
df_wiki_country_language_bronze.head(3)

,country,official_language,regional_language,minority_language,national_language,widely_spoken
0,abkhaziaa,abkhaz russian,,georgian,abkhaz,
1,afghanistan,pashto dari,uzbekb turkmenb pashaib nuristanib balochib pamirib,,pashto,
2,albania,albanian,,greek macedonian aromanian,,italian


#### Step8.  Fix casing of Country column <a class="anchor" id="S8"></a>

In [116]:
#Initcap each word in the country column
df_wiki_country_language_bronze['country']  = df_wiki_country_language_bronze.country.str.title()
df_wiki_country_language_bronze.head(3)

,country,official_language,regional_language,minority_language,national_language,widely_spoken
0,Abkhaziaa,abkhaz russian,,georgian,abkhaz,
1,Afghanistan,pashto dari,uzbekb turkmenb pashaib nuristanib balochib pamirib,,pashto,
2,Albania,albanian,,greek macedonian aromanian,,italian


#### Step9.  Fix null columns and data type after the new column additions <a class="anchor" id="S9"></a>

In [126]:
df_wiki_country_language_bronze.dtypes

country              object
official_language    object
regional_language    object
minority_language    object
national_language    object
widely_spoken        object
dtype: object

In [129]:
df_wiki_country_language_bronze['country'] = pd.Series(df_wiki_country_language_bronze['country'], dtype="string")
df_wiki_country_language_bronze['official_language'] = pd.Series(df_wiki_country_language_bronze['official_language'], dtype="string")
df_wiki_country_language_bronze['regional_language'] = pd.Series(df_wiki_country_language_bronze['regional_language'], dtype="string")
df_wiki_country_language_bronze['minority_language'] = pd.Series(df_wiki_country_language_bronze['minority_language'], dtype="string")
df_wiki_country_language_bronze['national_language'] = pd.Series(df_wiki_country_language_bronze['national_language'], dtype="string")
df_wiki_country_language_bronze['widely_spoken'] = pd.Series(df_wiki_country_language_bronze['widely_spoken'], dtype="string")

In [133]:
df_wiki_country_language_bronze.dtypes

country              string
official_language    string
regional_language    string
minority_language    string
national_language    string
widely_spoken        string
dtype: object

In [132]:
#Lets fill the NAs with an empty string '' for regional_language, minority_language, national_language, widely_spoken attribute
df_wiki_country_language_bronze[['regional_language']] = df_wiki_country_language_bronze[['regional_language']].fillna(value='')
df_wiki_country_language_bronze[['minority_language']] = df_wiki_country_language_bronze[['minority_language']].fillna(value='')
df_wiki_country_language_bronze[['national_language']] = df_wiki_country_language_bronze[['national_language']].fillna(value='')
df_wiki_country_language_bronze[['widely_spoken']] = df_wiki_country_language_bronze[['widely_spoken']].fillna(value='')

In [134]:
df_wiki_country_language_bronze.isnull().sum()

country              0
official_language    0
regional_language    0
minority_language    0
national_language    0
widely_spoken        0
dtype: int64

### Put Data (Silver) <a class="anchor" id="PS"></a>
- Raw data extraction for the file, API based and web datasets. Let us call this __Bronze Layer__.
- Data transformation using python from Raw to Processed stage. We will call this __Silver Layer__.
- Finally store the processed data using standard taxonomy in a SQL based serving layer. We will call this __Gold Layer__.

#### Write the transformed dataframe to the silver zone <a class="anchor" id="WS"></a>

In [135]:
silver_file_name = silver_data_folder + "/wiki_country_language.csv"
df_wiki_country_language_bronze.to_csv(silver_file_name, index=False)

#### Read silver zone country and language file <a class="anchor" id="RS"></a>

In [136]:
df_wiki_country_language_silver = pd.read_csv(silver_file_name)
df_wiki_country_language_silver.head(3).transpose()

,0,1,2
country,Abkhaziaa,Afghanistan,Albania
official_language,abkhaz russian,pashto dari,albanian
regional_language,NaN,uzbekb turkmenb pashaib nuristanib balochib pamirib,NaN
minority_language,georgian,NaN,greek macedonian aromanian
national_language,abkhaz,pashto,NaN
widely_spoken,NaN,NaN,italian
